In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [2]:
TARGET_COL = "nutriscore_letter"  

# 2) Tabular (numeric) feature columns (CHANGE THIS)
TABULAR_COLS = [
    'nova_group', 'fat_100g',
    'saturated_fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g',
    'proteins_100g', 'contains_palm_oil', 'vegetarian_status', 'vegan_status',
    'nutrient_level_fat', 'nutrient_level_saturated_fat',
    'nutrient_level_sugars', 'nutrient_level_salt', 'ecoscore_grade', 'ecoscore_score',
    'carbon_footprint_100g', 'additives_count', 'sugar_ratio',
    'energy_density', 'protein_ratio', 'macro_balance', 'healthy_score',
    'log_energy_kcal_100g', 'log_salt_100g'
]

# 3) Text columns (already preprocessed)
TEXT_COLS = [
    "brand_cleaned",
    "allergens_cleaned",
    "ingredients_text_cleaned",
    "countries_cleaned",
    "additives_cleaned",
]

In [4]:
df = pd.read_csv("FoodFactsCleaned.csv")

# Ensure all text cols exist and are strings
for c in TEXT_COLS:
    if c not in df.columns:
        raise ValueError(f"Missing text column: {c}")
    df[c] = df[c].fillna("").astype(str)

# Concatenate all text into a single field
df["text_concat"] = df[TEXT_COLS].fillna("").agg(" ".join, axis=1)

# Define X and y
X = df[["text_concat"] + TABULAR_COLS]
y = df[TARGET_COL]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train size:", len(X_train), "Test size:", len(X_test))

Train size: 2548 Test size: 637


In [5]:
preprocessor = ColumnTransformer(
    transformers=[
        ("text", TfidfVectorizer(
            ngram_range=(1, 2),
            min_df=5,
            max_features=30000
        ), "text_concat"),
        ("num", StandardScaler(with_mean=False), TABULAR_COLS),
    ],
    remainder="drop"
)

# Fit and transform directly
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

print("X_train_transformed shape:", X_train_transformed.shape)
print("X_test_transformed shape:", X_test_transformed.shape)

X_train_transformed shape: (2548, 4737)
X_test_transformed shape: (637, 4737)


In [ ]:
models = {
    "LogReg": LogisticRegression(
        max_iter=2000,
        n_jobs=-1,
        solver="saga",
        class_weight="balanced",
    ),

    "LinearSVC": LinearSVC(
        C=1.0,
        class_weight="balanced",
    ),

    "SGDClassifier (hinge SVM)": SGDClassifier(
        loss="hinge",
        max_iter=2000,
        class_weight="balanced",
        n_jobs=-1,
        random_state=42
    ),

}


In [7]:
models = {
    "LogReg": LogisticRegression(
        max_iter=2000,
        n_jobs=-1,
        solver="saga",
        class_weight="balanced",
    ),

    "LinearSVC": LinearSVC(
        C=1.0,
        class_weight="balanced",
    ),

    "SGDClassifier (hinge SVM)": SGDClassifier(
        loss="hinge",
        max_iter=2000,
        class_weight="balanced",
        n_jobs=-1,
        random_state=42
    ),

}


In [8]:
# =========================================
# 5. Train & evaluate each model (using pre-transformed data)
# =========================================

results = []

for name, clf in models.items():
    print(f"\n=== Training {name} ===")
    
    clf.fit(X_train_transformed, y_train)
    y_pred = clf.predict(X_test_transformed)

    acc = accuracy_score(y_test, y_pred)
    f1_macro = f1_score(y_test, y_pred, average="macro")

    print(f"{name}")
    print("-" * len(name))
    print(f"Accuracy:  {acc:.4f}")
    print(f"Macro F1:  {f1_macro:.4f}")
    print("\nClassification report:")
    print(classification_report(y_test, y_pred))

    results.append({
        "model": name,
        "accuracy": acc,
        "macro_f1": f1_macro
    })


=== Training LogReg ===


c:\Users\demer\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


LogReg
------
Accuracy:  0.8116
Macro F1:  0.7169

Classification report:
              precision    recall  f1-score   support

         1.0       0.98      0.87      0.92       176
         2.0       0.30      0.55      0.39        20
         3.0       0.62      0.71      0.66        69
         4.0       0.70      0.74      0.72       141
         5.0       0.92      0.86      0.89       231

    accuracy                           0.81       637
   macro avg       0.70      0.75      0.72       637
weighted avg       0.84      0.81      0.82       637


=== Training LinearSVC ===
LinearSVC
---------
Accuracy:  0.7771
Macro F1:  0.6514

Classification report:
              precision    recall  f1-score   support

         1.0       0.90      0.94      0.92       176
         2.0       0.38      0.30      0.33        20
         3.0       0.53      0.49      0.51        69
         4.0       0.63      0.62      0.62       141
         5.0       0.86      0.88      0.87       231

   

c:\Users\demer\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\svm\_base.py:1258: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
